<a href="https://colab.research.google.com/github/atulyadav27/MNIST_classification/blob/main/MNIST_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
from torchvision import datasets, transforms
transform = transforms.ToTensor()

train_dataset = datasets.MNIST(root = './data',train = True, download = True,transform = transform)
test_dataset = datasets.MNIST(root = './data',train = False, download = True,transform = transform)

In [ ]:
train_data,label = next(iter(train_dataset))
train_data.shape


In [ ]:
train_data_loader = torch.utils.data.DataLoader(dataset = train_dataset,batch_size = 32,shuffle = True)
test_data_loader = torch.utils.data.DataLoader(dataset = test_dataset, batch_size = 32,shuffle = False)


In [ ]:
train_data,label = next(iter(train_data_loader))
train_data[1].shape




#Visualize the data

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(train_data[0].squeeze(),cmap = "gray")
plt.title(label[0].item())
plt.axis(False)

In [ ]:
import torch
from torch import nn
torch.manual_seed(42)
class MNIST_classifier(nn.Module):
  def __init__(self,input_shape,hidden_units,output_shape):
    super().__init__()
    self.layer_stack = nn.Sequential(
        nn.Flatten(),
        nn.Linear(in_features = input_shape,out_features =hidden_units ),
        nn.ReLU(),
        nn.Linear(in_features = hidden_units, out_features = output_shape)

    )
  def forward(self,x):
    return self.layer_stack(x)



In [ ]:
#instantaniate the Model
import torch
torch.manual_seed(42)

Model_0 = MNIST_classifier(input_shape = 28*28,
                           hidden_units = 16,
                           output_shape = 10).to("cpu")
Model_0.state_dict()

In [ ]:
y_pred = Model_0(train_data).argmax(dim = 1)
y_pred


In [ ]:
label

In [ ]:
def accuracy_fn(y_true, y_pred):
  acc = torch.eq(y_true,y_pred).sum().item()
  acc /=len(y_true)
  acc *= 100
  return acc

In [ ]:
acc = accuracy_fn(y_true = label,

                  y_pred = y_pred)
acc

In [ ]:
#select the loss function and optimizer
loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(params = Model_0.parameters(),lr = 0.1)



#Train and Test the model

In [ ]:
from tqdm.auto import tqdm
torch.manual_seed(42)
Epochs = 5;
for Epoch in tqdm(range(Epochs)):
  Loss = 0
  train_acc = 0
  Model_0.train()
  for batch , (X,Y) in enumerate(train_data_loader):
    #Do the forward pass
    predict = Model_0(X)
    #Calculate the loss
    loss = loss_function(predict,Y)
    Loss +=loss
    train_acc += accuracy_fn(y_true = Y,y_pred = predict.argmax(dim = 1))
    #Set optimizer zero grad
    optimizer.zero_grad()
    #Backpropagation
    loss.backward()
    #Set the gradient descent
    optimizer.step()
    if(batch%400==0):
      print(f"Exploered till the batch :{batch}")
  Loss /= len(train_data_loader)
  train_acc /= len(train_data_loader)
  Model_0.eval()
  test_loss =0
  test_acc = 0
  with torch.inference_mode():
    for x,y in test_data_loader:
      test_pred = Model_0(x)
      test_loss += loss_function(test_pred,y)
      test_acc += accuracy_fn(y_true=y,y_pred=test_pred.argmax(dim = 1))
    test_loss /= len(test_data_loader)
    test_acc /= len(test_data_loader)

  print(f"loss:{Loss:.4f},Train Accuracy :{train_acc:.4f},Test Loss:{test_loss:.4f},Test Accuracy:{test_acc:.4f}")








